# Before Starting

Go to https://aistudio.google.com/app/apikey copy generative language client free tier API key

After you did that click the key icon at the left sidebar add your API key with GOOGLE_API_KEY as name and your API key as the value and enable notebook access


# Part 2: Gemini API & Function calling with Python

 Function calling lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with. Function calling lets you use functions as tools in generative AI applications, and you can define more than one function within a single request.

This notebook provides code examples to help you get started. The documentation's [quickstart](https://ai.google.dev/gemini-api/docs/function-calling#python) is also a good place to start understanding function calling.

## Setup

### Install dependencies

In [4]:
%pip install -qU 'google-genai>=1.0.0'

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

In [5]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

Function calling should work with all the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) models with the GenAI SDK. It also works with the 1.5 generation of models.

In [8]:
MODEL_ID="gemini-2.0-flash-lite" # @param ["gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-03-25", "gemini-2.0-flash", "gemini-2.0-flash-lite"] {"allow-input":true, isTemplate: true}

## Setting up Functions as Tools

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`genai.types.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`


**Example: Lighting System Functions**

Here are 3 functions controlling a hypothetical lighting system. Note the docstrings and type hints.

In [9]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")

light_controls = [enable_lights, set_light_color, stop_lights]
instruction = """
  You are a helpful lighting system bot. You can turn
  lights on and off, and you can set the color. Do not perform any
  other tasks.
"""

## Basic Function Calling with Chat

Function calls naturally fit into multi-turn conversations. The Python SDK's `ChatSession (client.chats.create(...))` is ideal for this, as it automatically handles conversation history.

Furthermore, `ChatSession` simplifies function calling execution via its `automatic_function_calling` feature (enabled by default), which will be explored more later. For now, let's see a basic interaction where the model decides to call a function.

In [10]:
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": light_controls,
        "system_instruction": instruction,
    }
)

response = chat.send_message("It's awful dark in here...")

print(response.text)

LIGHTBOT: Lights enabled.
Alright, I've turned on the lights.



## Examining Function Calls and Execution History

To understand what happened in the background, you can examine the chat history.

The `Chat.history` property stores a chronological record of the conversation between the user and the Gemini model. You can get the history using `Chat.get_history()`. Each turn in the conversation is represented by a `genai.types.Content` object, which contains the following information:

**Role**: Identifies whether the content originated from the "user" or the "model".

**Parts**: A list of genai.types.Part objects that represent individual components of the message. With a text-only model, these parts can be:

* **Text**: Plain text messages.
* **Function Call (genai.types.FunctionCall)**: A request from the model to execute a specific function with provided arguments.
* **Function Response (genai.types.FunctionResponse)**: The result returned by the user after executing the requested function.


In [ ]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

It's awful dark in here...

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' }
--------------------------------------------------------------------------------


###user:

Function response: { will_continue=None scheduling=None id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

Ok, I turned the lights on.


--------------------------------------------------------------------------------


This history shows the flow:

1. **User**: Sends the message.

2. **Model**: Responds not with text, but with a `FunctionCall` requesting `enable_lights`.

3. **User (SDK)**: The `ChatSession` automatically executes `enable_lights()` because `automatic_function_calling` is enabled. It sends the result back as a `FunctionResponse`.

4. **Model**: Uses the function's result ("Lights enabled.") to formulate the final text response.

## Automatic Function Execution (Python SDK Feature)

As demonstrated above, the `ChatSession` in the Python SDK has a powerful feature called Automatic Function Execution. When enabled (which it is by default), if the model responds with a FunctionCall, the SDK will:

1. Find the corresponding Python function in the provided `tools`.

2. Execute the function with the arguments provided by the model.

3. Send the function's return value back to the model in a `FunctionResponse`.

4. Return only the model's final response (usually text) to your code.

This significantly simplifies the workflow for common use cases.

**Example: Math Operations**

In [14]:
from google.genai import types # Ensure types is imported

def add(a: float, b: float):
    """returns a + b."""
    return a + b

def subtract(a: float, b: float):
    """returns a - b."""
    return a - b

def multiply(a: float, b: float):
    """returns a * b."""
    return a * b

def divide(a: float, b: float):
    """returns a / b."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b

operation_tools = [add, subtract, multiply, divide]

chat = client.chats.create(
    model=MODEL_ID,
    config={
        "automatic_function_calling": {"disable": False} # Enabled by default
    }
)

response = chat.send_message(
    "I have 5567 cats, each owns 443434 mittens, how many mittens is that in total?"
)

print(response.text)

Let $C$ be the number of cats, which is 5567.
Let $M$ be the number of mittens each cat owns, which is 443434.
We want to find the total number of mittens. To do this, we multiply the number of cats by the number of mittens each cat owns.
Total number of mittens = $C \times M$
Total number of mittens = $5567 \times 443434$
To calculate this, we can perform the multiplication:
$5567 \times 443434 = 2468923858$
So, the total number of mittens is 2,468,923,858.

We can calculate the product as follows:
\begin{array}{@{}c@{\,}c@{}c@{}c@{}c@{}c@{}c@{}c@{}c@{}c}
& & & & 4 & 4 & 3 & 4 & 3 & 4 \\
& & & \times & & 5 & 5 & 6 & 7 \\
\hline
& & & 3 & 1 & 0 & 4 & 0 & 3 & 8 \\
& & 2 & 6 & 6 & 0 & 6 & 0 & 4 & 0 \\
& 2 & 2 & 1 & 7 & 1 & 7 & 0 & 0 & \\
2 & 2 & 1 & 7 & 1 & 7 & 0 & 0 & 0 & 0 \\
\hline
2 & 4 & 6 & 8 & 9 & 2 & 3 & 8 & 5 & 8 \\
\end{array}
The total number of mittens is 2,468,923,858.

Final Answer: The final answer is $\boxed{2468923858}$


Automatic execution handled the `multiply` call seamlessly.

## Automatic Function Schema Declaration

A key convenience of the Python SDK is its ability to automatically generate the required `FunctionDeclaration` schema from your Python functions. It inspects:

- **Function Name**: (`func.__name__`)

- **Docstring**: Used for the function's description.

- **Parameters**: Names and type annotations (`int`, `str`, `float`, `bool`, `list`, `dict`). Docstrings for parameters (if using specific formats like Google style) can also enhance the description.

- **Return Type Annotation**: Although not strictly used by the model for deciding which function to call, it's good practice.

You generally don't need to create `FunctionDeclaration` objects manually when using Python functions directly as tools.

However, you can generate the schema explicitly using `genai.types.FunctionDeclaration.from_callable` if you need to inspect it, modify it, or use it in scenarios where you don't have the Python function object readily available.

In [16]:
import json

set_color_declaration = types.FunctionDeclaration.from_callable(
    callable = set_light_color,
    client = client
)

print(json.dumps(set_color_declaration.to_json_dict(), indent=4))

{
    "description": "Set the light color. Lights must be enabled for this to work.",
    "name": "set_light_color",
    "parameters": {
        "properties": {
            "rgb_hex": {
                "type": "STRING"
            }
        },
        "required": [
            "rgb_hex"
        ],
        "type": "OBJECT"
    }
}


## Parallel function calls

The Gemini API can call multiple functions in a single turn. This caters for scenarios where there are multiple function calls that can take place independently to complete a task.

First set the tools up. Unlike the movie example above, these functions do not require input from each other to be called so they should be good candidates for parallel calling.

In [17]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

house_fns = [power_disco_ball, start_music, dim_lights]

Now call the model with an instruction that could use all of the specified tools.

In [18]:
# You generally set "mode": "any" to make sure Gemini actually *uses* the given tools.
party_chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": house_fns,
        "tool_config" : {
            "function_calling_config": {
                "mode": "any"
            }
        }
    }
)

# Call the API
response = party_chat.send_message(
    "Turn this place into a party!"
)


print_history(party_chat)

Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120
Disco ball is spinning!
Lights are now set to 20%
Starting music! energetic=True loud=True, bpm=120


NameError: name 'print_history' is not defined

Notice the single model turn contains three FunctionCall parts, which the SDK then executed before getting the final text response.

## Compositional Function Calling
The model can chain function calls across multiple turns, using the result from one call to inform the next. This allows for complex, multi-step reasoning and task completion.

**Example: Finding Specific Movie Showtimes**

Let's reuse the theater_functions and ask a more complex query that requires finding movies first, then potentially theaters, then showtimes.

In [20]:
import os

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY2')

# Example Functions
def get_weather_forecast(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_weather_forecast(location={location})")
    # TODO: Make API call
    print("Tool Response: {'temperature': 25, 'unit': 'celsius'}")
    return {"temperature": 16, "unit": "celsius"}  # Dummy response

def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")

    print("Tool Response: {'status': 'success'}")
    return {"status": "success"}

client = genai.Client(api_key = GOOGLE_API_KEY)
config = types.GenerateContentConfig(
    tools=[get_weather_forecast, set_thermostat_temperature]
)


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="If it's warmer than 20°C in London, set the thermostat to 20°C, otherwise set it to 18°C.",
    config=config,
)


print(response.text)

Tool Call: get_weather_forecast(location=London)
Tool Response: {'temperature': 25, 'unit': 'celsius'}
Tool Call: set_thermostat_temperature(temperature=18)
Tool Response: {'status': 'success'}
OK, I've set the thermostat to 18°C.



Here you can see that the model made conditional calls.


In [22]:

import re
from IPython.display import display
import ipywidgets as widgets
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY2')

chat_output = widgets.Output(
    layout=widgets.Layout(border="1px solid #ccc", height="320px", overflow="auto")
)
user_input = widgets.Text(
    placeholder="Type your message and press Enter…",
    layout=widgets.Layout(flex="1 1 auto")
)
send_button = widgets.Button(description="Send", button_style="primary")

SVG_TEMPLATE = (
    "<div style='display:flex;flex-direction:column;align-items:center;justify-content:center;height:100%;'>"
    "<svg width='120' height='120' viewBox='0 0 24 24' xmlns='http://www.w3.org/2000/svg'>"
    "  <path d='M12 2c-4 0-7 3-7 7 0 3 2 4 2 7h10c0-3 2-4 2-7 0-4-3-7-7-7z' fill='{color}' stroke='#333' stroke-width='0.5'/>"
    "  <rect x='9' y='18' width='6' height='2' fill='{color}' />"
    "  <rect x='10.5' y='21' width='3' height='2' fill='{color}' />"
    "</svg>"
    "<span style='margin-top:8px;font-size:14px;'>{msg}</span>"
    "</div>"
)

bulb_output = widgets.HTML(
    value=SVG_TEMPLATE.format(color="#666", msg="The light is OFF."),
    layout=widgets.Layout(border="1px solid #ccc", width="220px", height="320px")
)

# ---------------- State ----------------
light_is_on = False

# Default ON colour (Gold) and palette
COLOR_MAP = {
    "gold": "#ffd700",
    "warm": "#ffcc66",
    "white": "#ffffe0",
    "blue": "#40a0ff",
    "green": "#4caf50",
    "red": "#ff5252",
    "purple": "#ba68c8",
}
current_on_color = COLOR_MAP["gold"]

# ---------------- Helper ----------------

def _update_bulb(is_on: bool, msg: str):
    """Re‑render bulb in its current colour plus caption."""
    global light_is_on
    light_is_on = is_on
    color = current_on_color if is_on else "#666"  # dark grey when off
    bulb_output.value = SVG_TEMPLATE.format(color=color, msg=msg)

# ---------------- Tool Functions ----------------

def turn_on_lights():
    if light_is_on:
        _update_bulb(True, "The lights were already ON.")
        return {"status": "The lights were already ON."}
    _update_bulb(True, "The lights have been turned ON.")
    return {"status": "The lights have been turned ON."}


def turn_off_lights():
    if not light_is_on:
        _update_bulb(False, "The lights were already OFF.")
        return {"status": "The lights were already OFF."}
    _update_bulb(False, "The lights have been turned OFF.")
    return {"status": "The lights have been turned OFF."}


def set_light_color(color: str):
    """Change the ON colour. Accepts common names (gold, blue, …) or hex (#RRGGBB)."""
    global current_on_color

    color_str = color.strip().lower()
    if color_str in COLOR_MAP:
        current_on_color = COLOR_MAP[color_str]
        status = f"Colour set to {color_str}."
    elif re.fullmatch(r"#?[0-9a-f]{6}", color_str):
        current_on_color = "#" + color_str.lstrip("#")
        status = f"Custom colour {current_on_color} applied."
    else:
        status = "Invalid colour. Available names: " + ", ".join(COLOR_MAP.keys())
        return {"status": status}

    # If light is ON, re‑render with new colour immediately
    _update_bulb(light_is_on, status)
    return {"status": status}

# ---------------- Gemini 2.0 Flash Setup ----------------
light_controls = [turn_on_lights, turn_off_lights, set_light_color]
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",  # ← upgraded per request
    tools=light_controls,
)
chat = model.start_chat(enable_automatic_function_calling=True)

# ---------------- Chat Handling ----------------

def _append_chat(role: str, text: str):
    with chat_output:
        print(f"{role}: {text}\n")

@chat_output.capture()
def _send(_=None):
    prompt = user_input.value.strip()
    if not prompt:
        return
    _append_chat("👤 You", prompt)
    user_input.value = ""
    try:
        response = chat.send_message(prompt)
        _append_chat("🤖 Gemini", response.text)
    except Exception as e:
        _append_chat("⚠️ Error", str(e))

# ---------------- Bind UI ----------------
send_button.on_click(_send)
user_input.on_submit(_send)

ui = widgets.HBox([
    widgets.VBox([chat_output, widgets.HBox([user_input, send_button])], layout=widgets.Layout(flex="1 1 0%")),
    bulb_output
])

display(ui)



# Exercises

In [ ]:
# ——— solution functions ——— define your functions here


calc_tools = [add, multiply]
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": calc_tools,
        "system_instruction": instruction,
    }
)

response = chat.send_message("What’s 7 × 8?  Also, what’s 5 + 4?")

print(response.text)


7 times 8 is 56, and 5 plus 4 is 9.



In [ ]:
# ╔═══════════════ Exercise 2 — Temperature Converter (SOLVED) ═══════════════╗
import google.generativeai as genai
from google.generativeai import types

# ——— solution functions ——— define your functions here


temp_tools = [f_to_c, c_to_f]
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": temp_tools,
        "system_instruction": instruction,
    }
)

response = chat.send_message("Convert 72 °F to Celsius.")

print(response.text)


72 degrees Fahrenheit is 22.2 degrees Celsius.



In [ ]:
# ╔═══════════════ Exercise 3 — String Helper (SOLVED) ═══════════════╗
import google.generativeai as genai
from google.generativeai import types

# ——— solution functions ——— define your functions here

# ——— Gemini wiring & quick demo ———
string_tools = [reverse_str, count_vowels]
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": string_tools,
        "system_instruction": instruction,
    }
)


response = chat.send_message("How many vowels are in 'function calling'? Also give me the text reversed.")

print(response.text)


There are 5 vowels in 'function calling'. The reversed text is 'gnillac noitcnuf'.



# ╔═════════ Best-Practice Cheat-Sheet ═══════════╗

### Best-Practice Checklist for Gemini Function Calling 🔧

| Area | What to Do | Why It Matters |
|------|------------|----------------|
| **Function & parameter descriptions** | • Write *explicit, unambiguous* docstrings.<br>• Spell out units, ranges, and edge cases. | The model chooses the tool—and its arguments—based on these strings alone. |
| **Naming** | • Use descriptive snake-case names: `add_user`, `fetch_fx_rate`.<br>• Avoid spaces, periods, or dashes. | Clear names lower the odds of the wrong tool being picked. |
| **Strong typing** | • Annotate every parameter (`int`, `str`, etc.).<br>• For limited choices, declare an **enum**. | Tighter schemas ⇒ fewer invalid calls. |
| **Tool selection** | • Keep the *active* tool set ≤ 10-20.<br>• Dynamically load only the tools relevant to the current task. | Too many tools confuses the model and slows responses. |
| **Prompt engineering** | 1. **Role** – e.g. “You are a helpful weather assistant.”<br>2. **Instructions** – e.g. “Never guess dates; always call the forecast API.”<br>3. **Clarification** – e.g. “If location is unclear, ask a follow-up question.” | Gives the model a deterministic decision path. |
| **Temperature** | • Use `temperature = 0` for deterministic, reproducible calls. | Higher values introduce randomness and break workflows. |
| **Validation** | • For high-stakes actions (orders, payments), echo the function call back to the user for confirmation **before** executing. | Prevents costly mistakes. |
| **Error handling** | • Catch API failures and bad inputs.<br>• Return structured errors, e.g. `{"error": "City not found"}`. | Lets the model apologise or re-ask instead of hallucinating. |
| **Security** | • Keep credentials in env vars or a secrets manager.<br>• Never log sensitive parameters. | Reduces attack surface and data leaks. |
| **Token limits** | • Schemas and descriptions count toward the token budget.<br>• If you’re near the limit, shorten docstrings or split tasks into smaller tool sets. | Prevents truncation and “too many tokens” errors. |

> **Rule of thumb:** concise, strongly-typed tool specs + low-temperature prompts = predictable, safe function calling.
